# Data Preprocessing
* change to space splitted basis.
* remove duplicate rows and rows that consist too few element.

# Train Gensim Models - Word2Vec

In [ ]:
from gensim import corpora
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
from pprint import pprint
import gensim
import gensim.downloader as api
import numpy as np
import pandas as pd

In [ ]:
# train word2vec model by genism, with data: text8
dataset = api.load('text8')
data = [d for d in dataset]

# train word2vec model. Default result vector size = 100
model = Word2Vec(data, min_count=0, workers=cpu_count())

model.save('./gensim_models/word2vec_model_text8')

In [ ]:
# update our model with tweet dataset

# create copora dictionary
tweet_data = pd.read_csv('./datasets/tweet_stopwords_duplicates_short_removed_space_splitted.csv')
data_list = tweet_data.iloc[:, 0].tolist()
# tokenize data into words
texts = [[text for text in tweet.split()] for tweet in data_list]
# create dictionary by copora
dictionary = corpora.Dictionary(texts)
print(type(dictionary))
print(type(texts))


# load text8 word2vec dataset
model = Word2Vec.load('./gensim_models/word2vec_model_text8')
print(model['media'], '\n')

# update the model with tweets dataset
model.build_vocab(texts, update=True)
model.train(data, total_examples=model.corpus_count, epochs=model.iter)
print(model['media'])

# save the model
model.save('./gensim_models/word2vec_model_text8_tweet')

In [ ]:
# train a word2vec model with tweet dataset only

# create copora dictionary
tweet_data = pd.read_csv('./datasets/tweet_stopwords_duplicates_short_removed_space_splitted.csv')
data_list = tweet_data.iloc[:, 0].tolist()
# tokenize data into words
texts = [[text for text in tweet.split()] for tweet in data_list]
# create dictionary by copora
dictionary = corpora.Dictionary(texts)
print(type(dictionary))
print(type(texts))

# train word2vec model. Default result vector size = 100
model = Word2Vec(texts, min_count=0, workers=cpu_count())
print(model['trump'], '\n')
print(model['taiwan'])

model.save('./gensim_models/word2vec_model_tweet')

In [ ]:
# check the result with only tweets data
print(model.wv.most_similar('trump'), '\n')
print(model.wv.most_similar('taiwan'), '\n')
print(model.wv.most_similar('obama'), '\n')
print(model.wv.most_similar('freak'), '\n')

# Gensim Training - Doc2Vec

In [ ]:
import gensim
import gensim.downloader as api
import numpy as np
import pandas as pd

In [ ]:
# function to create 'tagged document'
def create_tagged_document(list_of_list_of_word):
    for i, list_of_word in enumerate(list_of_list_of_word):
        yield gensim.models.doc2vec.TaggedDocument(list_of_word, [i])

In [ ]:
# create 'tagged document' dataset of text8
# load text8 dataset
dataset = api.load('text8')
data = [d for d in dataset]

# create tagged document of text8
training_data_text8 = list(create_tagged_document(data))

In [ ]:
# initialize doc2vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

# build the vocabulary
model.build_vocab(training_data_text8)

# train the Doc2Vec model
model.train(training_data_text8, total_examples=model.corpus_count, epochs=model.epochs)

model.save('./gensim_models/doc2vec_model_text8')

In [ ]:
# create a dataset with text8 and tweets data
tweet_data = pd.read_csv('./datasets/tweet_stopwords_duplicates_short_removed_space_splitted.csv')
data_list = tweet_data.iloc[:, 0].tolist()

# tokenize data into words
training_tweet = [[text for text in tweet.split()] for tweet in data_list]

# create tagged document with text8 and tweet
training_data_text8_tweet = list(create_tagged_document(data + training_tweet))

# create tagged document with tweet only
training_data_tweet = list(create_tagged_document(training_tweet))

In [ ]:
# create a model for both text8 and tweets data
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

# build the vocabulary
model.build_vocab(training_data_text8_tweet)

# train the model
model.train(training_data_text8_tweet, total_examples=model.corpus_count, epochs=model.epochs)

model.save('./gensim_models/doc2vec_model_text8_tweet')

# Create Tagged dataset and gensim models

In [1]:
import gensim
import gensim.downloader as api
import numpy as np
import pandas as pd

In [2]:
# function to create 'tagged document'
def create_tagged_document(list_of_list_of_word):
    for i, list_of_word in enumerate(list_of_list_of_word):
        yield gensim.models.doc2vec.TaggedDocument(list_of_word, [i])

In [15]:
# prepare data by concating original and removed stopwords together
original = pd.read_csv("./datasets/8_original_text_only_stopwords_removed.csv", sep='\t')
retweet = pd.read_csv("./datasets/7_retweet_text_only_stopwords_removed.csv", sep='\t')

print(len(original))
print(original.columns)
print(len(retweet))
print(retweet.columns)

30567
Index(['is_RT', 'retweet_from', 'whole_tweet_text', 'original_text',
       'retweet_text', 'stopwords_removed_original_text'],
      dtype='object')
614
Index(['is_RT', 'retweet_from', 'whole_tweet_text', 'original_text',
       'retweet_text', 'stopwords_removed_retweet_text'],
      dtype='object')


In [5]:
# tokenize data into words
training_tweet = [[text for text in str(tweet).split()] for tweet in retweet.loc[:, 'stopwords_removed_retweet_text']]
# training_tweet = [[text for text in str(tweet).split()] for tweet in original.loc[:, 'stopwords_removed_original_text']]

# create tagged document with tweet only
training_data_tweet = list(create_tagged_document(training_tweet))

# create a model for only tweets
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

# build the vocabulary
model.build_vocab(training_data_tweet)

# train the model
model.train(training_data_tweet, total_examples=model.corpus_count, epochs=model.epochs)

model.save('./gensim_models/doc2vec_model_retweet')
# model.save('./gensim_models/doc2vec_model_original')

# Create dataset with Doc2Vec Features

In [16]:
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm

In [17]:
model = gensim.models.doc2vec.Doc2Vec.load('./gensim_models/doc2vec_model_retweet')

for i, j in enumerate(tqdm(retweet.loc[:, 'stopwords_removed_retweet_text'])):
    element_splitted = str(j).split(" ")
    feature_array = model.infer_vector(element_splitted)
    for a, b in enumerate(feature_array):
        column_name = 'feature_' + str(a)
        retweet.loc[i, column_name] = b

print(retweet.head())
print(retweet.describe())
retweet.to_csv('./datasets/10_retweet_text_doc2vec.csv', sep='\t', header=True, index=None)

100%|██████████| 614/614 [00:11<00:00, 55.34it/s]


   is_RT     retweet_from                                   whole_tweet_text  \
0      1    SteveSGoddard  RT @SteveSGoddard: The @nytimes was thrilled b...   
1      1   DonaldJTrumpJr  RT @DonaldJTrumpJr: Happy new year everyone. #...   
2      1  VoteHillary2016  RT @VoteHillary2016: Trump says Taiwain leader...   
3      1   NeonKnight1337  RT @NeonKnight1337: Donald Trump's call with T...   
4      1    gatewaypundit  RT @gatewaypundit: The Trump Hotel Waikiki loo...   

  original_text                                       retweet_text  \
0           NaN   The @nytimes was thrilled by Obama talking to...   
1           NaN   Happy new year everyone. #newyear #family #va...   
2           NaN   Trump says Taiwain leader called him, though ...   
3           NaN   Donald Trump's call with Taiwan president was...   
4           NaN   The Trump Hotel Waikiki looks like a lovely r...   

                      stopwords_removed_retweet_text  feature_0  feature_1  \
0  nytimes thrilled 

In [18]:
model = gensim.models.doc2vec.Doc2Vec.load('./gensim_models/doc2vec_model_original')

for i, j in enumerate(tqdm(original.loc[:, 'stopwords_removed_original_text'])):
    element_splitted = str(j).split(" ")
    feature_array = model.infer_vector(element_splitted)
    for a, b in enumerate(feature_array):
        column_name = 'feature_' + str(a)
        original.loc[i, column_name] = b

print(original.head())
print(original.describe())
original.to_csv('./datasets/10_original_text_doc2vec.csv', sep='\t', header=True, index=None)

100%|██████████| 30567/30567 [14:47<00:00, 34.42it/s]


   is_RT  retweet_from                                   whole_tweet_text  \
0      0           NaN  "@MysticWolf12001: @realDonaldTrump C'mon, you...   
1      0           NaN  "@Chad_Williams91: @realDonaldTrump if you're ...   
2      0           NaN  "@HunterBalthazor: @realDonaldTrump if you ran...   
3      0           NaN  "@kyleraccio: @realDonaldTrump @Vinny_Titone I...   
4      0           NaN                @HarryCraig96 @TrumpTowerNY Thanks!   

                                       original_text  retweet_text  \
0  "@MysticWolf12001: @realDonaldTrump C'mon, you...           NaN   
1  "@Chad_Williams91: @realDonaldTrump if you're ...           NaN   
2  "@HunterBalthazor: @realDonaldTrump if you ran...           NaN   
3  "@kyleraccio: @realDonaldTrump @Vinny_Titone I...           NaN   
4                @HarryCraig96 @TrumpTowerNY Thanks!           NaN   

                     stopwords_removed_original_text  feature_0  feature_1  \
0           MysticWolf12001 Cmon one b